In [ ]:
from openai import AzureOpenAI 

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
import os
client = AzureOpenAI(api_key=os.getenv("AZURE_OPENAI_KEY"), 
                     api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
                     azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
                     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"))




print("************")

api_key=os.getenv("AZURE_OPENAI_KEY")
print(f"************api_key= {api_key}")

completion = client.chat.completions.create(
    
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

In [9]:
import pandas as pd
eval_res = pd.DataFrame(columns=["question", "ground_truth", "context", "response", "groundeness", "relevance", "similarity", "coherence"])
row = pd.DataFrame([{"question": "Write a haiku about recursion in programming.", "ground_truth": "Recursion in code, a loop that repeats itself, until it doesn't."}])   
pd.concat([eval_res, row], ignore_index=True)

,question,ground_truth,context,response,groundeness,relevance,similarity,coherence
0,Write a haiku about recursion in programming.,"Recursion in code, a loop that repeats itself,...",NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
import pandas as pd

eval_res = pd.DataFrame(columns=["question", "ground_truth", "context", "response"])

eval_res["question"] = ["Write a haiku about recursion in programming."]
eval_res["ground_truth"] = ["Recursion in code, a loop that repeats itself, until it doesn't."]
eval_res["context"] = ["You are a helpful assistant."]
eval_res["response"] = ["Recursion in code, a loop that repeats itself, until it doesn't."]

eval_res_scores = pd.DataFrame(columns=["question", "ground_truth", "context", "response"])


def groundeness(x):
    return 0.5
def relevance(x):
    return 0.4

evaluators = {"groundeness": groundeness, "relevance": relevance}
scores = {}

for index, row in eval_res.iterrows():
    
    new_row = row.to_dict()
    
    for name, func in evaluators.items():
        new_row[name] = func(row)
        if name not in scores:
            scores[name] = new_row[name]
        else:
            scores[name] += new_row[name]

    eval_res_scores = pd.concat([eval_res_scores, pd.DataFrame([new_row])], ignore_index=True)

eval_res_scores.head()
print(scores)

d = eval_res_scores.to_dict(orient="records")
print(d) 

                        

{'groundeness': 0.5, 'relevance': 0.4}
[{'question': 'Write a haiku about recursion in programming.', 'ground_truth': "Recursion in code, a loop that repeats itself, until it doesn't.", 'context': 'You are a helpful assistant.', 'response': "Recursion in code, a loop that repeats itself, until it doesn't.", 'groundeness': 0.5, 'relevance': 0.4}]


In [5]:
scores = {}
#add code which add a new score to the scores dictionary. The key like relevancy could new (when we create it a first time)
#check if a keys exists in a dictionary if yes , just expand the list with a new value otherwise create a new key with a value 

def add_score(scores, key, value):
    if key in scores:
        scores[key].extend([value])
    else:
        scores[key] = [value]
        
add_score(scores, "relevancy", 0.5)


In [6]:
scores

{'relevancy': [0.5]}

In [7]:
add_score(scores, "relevancy", 0.1)

In [8]:
scores

{'relevancy': [0.5, 0.1]}

In [3]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
import os

In [4]:
model_config = {
    "azure_endpoint": os.getenv("AZURE_OPENAI_EVALUATION_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "azure_deployment": os.getenv("AZURE_OPENAI_EVALUATION_DEPLOYMENT"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
}

print (model_config)

{'azure_endpoint': 'https://openai-australia-east-303474.openai.azure.com/openai/deployments/gpt-4o-3/chat/completions?api-version=2024-08-01-preview', 'api_key': '92ab6cf3f727424aad3e5b8b1ae55030', 'azure_deployment': 'gpt-4o-3', 'api_version': '2024-08-01-preview'}


In [5]:
from azure.ai.evaluation import (
    RelevanceEvaluator,
    GroundednessEvaluator,
    SimilarityEvaluator,
    CoherenceEvaluator,
)

In [8]:
groundeness = GroundednessEvaluator(model_config)

response=groundeness( response="Microsoft Fabric is an all-in-one analytics solution for enterprises. It provides built-in enterprise-grade governance and compliance capabilities, powered by Microsoft Purview. Fabric allows users to bring their existing Azure Data Factory to their Fabric workspace, and it supports various certifications such as HIPAA, ISO 27017, ISO 27018, ISO 27001, and ISO 27701. For more information on the latest features and updates, you can follow the Microsoft Fabric Updates Blog and the Microsoft Fabric Blog",
        context="Microsoft Fabric is an end-to-end analytics and data platform designed for enterprises that require a unified solution. It encompasses data movement, processing, ingestion, transformation, real-time event routing, and report building. It offers a comprehensive suite of services including Data Engineering, Data Factory, Data Science, Real-Time Analytics, Data Warehouse, and Databases. With Fabric, you don't need to assemble different services from multiple vendors. Instead, it offers a seamlessly integrated, user-friendly platform that simplifies your analytics requirements. Operating on a Software as a Service (SaaS) model, Fabric brings simplicity and integration to your solutions.")["gpt_groundedness"]

response


1.0